In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import datetime,timedelta
import re

In [2]:
kdd_root = '/home/chenhr/Documents/KDD2017/dataSets/dataSets/dataSet_phase2/'
volume_data = 'volume(table 6)_training2.csv'
weather_root = '/home/chenhr/Documents/KDD2017/dataSets/dataSets/testing_phase1/'
weather_data = 'weather (table 7)_test1.csv'


In [3]:
volume_raw_test_data = pd.read_csv(kdd_root+volume_data)
volume_raw_test_data.shape

(128823, 6)

In [4]:
#do not need in this data
#reform the date
def reform_date(date):
    _date_ = re.split('\/',date)
    if len(_date_[1])==1:
        _date_[1]='0'+_date_[1]
    if len(_date_[2])==1:
        _date_[2]='0'+_date_[2]
        
    return _date_[0]+'-'+_date_[1]+'-'+_date_[2]

In [5]:
weather_raw_data = pd.read_csv(weather_root+weather_data)
#do not need in this data
#weather_raw_data['date']=weather_raw_data['date'].apply(reform_date)

weather_raw_data.shape

(56, 9)

In [6]:
volume_raw_test_data['veh_type'] = volume_raw_test_data['veh_type'].fillna(2)
volume_raw_test_data.head()

,date_time,tollgate,direction,model,is_etc,veh_type
0,2016-10-18 23:55:58,3,0,1,1,2.0
1,2016-10-18 23:56:40,3,0,1,0,2.0
2,2016-10-18 23:56:52,3,0,3,0,2.0
3,2016-10-18 23:57:14,3,0,1,0,2.0
4,2016-10-18 23:57:38,3,0,1,0,2.0


In [7]:
#Creat time window
def get_time_window(time):
    time = datetime.strptime(time,"%Y-%m-%d %H:%M:%S")
    time_window_minute = int(math.floor(time.minute/20)*20)
    time_window_start = datetime(time.year,time.month,time.day,time.hour,
                                 time_window_minute,0)
    time_window_end = time_window_start + timedelta(minutes=20)
    time_window ='[' + str(time_window_start) + ',' + str(time_window_end) +")"
    return time_window

In [8]:
#Creat time hour
def get_time_hour(time):
    time = datetime.strptime(time,"%Y-%m-%d %H:%M:%S")
    time_hour = int(math.floor(time.hour/3)*3)
    return time_hour


In [9]:
volume_raw_test_data['time_window'] = volume_raw_test_data['date_time'].apply(get_time_window)
volume_raw_test_data['time_hour'] = volume_raw_test_data['date_time'].apply(get_time_hour)


In [10]:
volume_raw_test_data.shape


(128823, 8)

In [11]:
time_window_set = set(volume_raw_test_data.time_window.values)
ID = [1,2,3]
DIRECTION =[0,1]
len(time_window_set)

504

In [12]:
volume_data_columns = ['time_window','tollgate','direction','model0',
                       'model1','model2','model3',
                       'model4','model5','model6',
                       'has_etc','no_etc','veh_type0','veh_type1',
                       'veh_type2','pressure','sea_pressure','wind_direction',
                       'wind_speed','temperature','rel_humidity','precipitation',
                       'weekday','date','hour','volumes']
volume_collected_test_data = pd.DataFrame(columns=[volume_data_columns])
#print volume_collected_training_data

In [13]:
def get_wd(wind_direction):
    wd_ =np.arange(0,371.25,22.5)
    if wind_direction != 999017:
        temp_list = [math.fabs(wind_direction-wd) for wd in wd_]
        wd_index = temp_list.index(min(temp_list))
    else:
        return 'no_direction'
      
    switcher ={
        0:'N',
        1:'NNE',
        2:'NE',
        3:'ENE',
        4:'E',
        5:'ESE',
        6:'SE',
        7:'SSE',
        8:'S',
        9:'SSW',
        10:'SW',
        11:'WSW',
        12:'W',
        13:'WNW',
        14:'NW',
        15:'NNW',
        16:'N',
    }
    return switcher.get(wd_index,'no_direction')
   
    

In [14]:
volume_raw_test_data.head()

,date_time,tollgate,direction,model,is_etc,veh_type,time_window,time_hour
0,2016-10-18 23:55:58,3,0,1,1,2.0,"[2016-10-18 23:40:00,2016-10-19 00:00:00)",21
1,2016-10-18 23:56:40,3,0,1,0,2.0,"[2016-10-18 23:40:00,2016-10-19 00:00:00)",21
2,2016-10-18 23:56:52,3,0,3,0,2.0,"[2016-10-18 23:40:00,2016-10-19 00:00:00)",21
3,2016-10-18 23:57:14,3,0,1,0,2.0,"[2016-10-18 23:40:00,2016-10-19 00:00:00)",21
4,2016-10-18 23:57:38,3,0,1,0,2.0,"[2016-10-18 23:40:00,2016-10-19 00:00:00)",21


In [16]:
for _time_window_set_ in time_window_set:
    time_window_data = volume_raw_test_data[volume_raw_test_data.time_window
                                               == _time_window_set_]
    _date_ = re.split('\[|\ |\,|\)',_time_window_set_)[1]
    get_time = re.split('\[|\ |\,|\)',_time_window_set_)[2]
    time_ = datetime.strptime(get_time,'%H:%M:%S')
    hour_ = int(math.floor(time_.hour/3)*3)
    #month_,date_ = [re.split('\-',_date_)[1],re.split('\-',_date_)[2]]
    time_window_ = '[' + re.split('\[|\ |\,|\)',_time_window_set_)[2] + ','+re.split('\[|\ |\,|\)',_time_window_set_)[4]+')'
    weekday_ = datetime.strptime(_date_,'%Y-%m-%d').weekday()
    temp_weather = weather_raw_data[(weather_raw_data['date']==_date_)
                                    &(weather_raw_data['hour']==hour_)]
   
    for _id in ID:
        for _direction in DIRECTION:
            if _id==2 and _direction==1:
                continue
            
            temp_s = pd.Series(np.nan,index=volume_data_columns)
            
            temp_time_window_data = time_window_data[(time_window_data.tollgate == _id)
                                                     &(time_window_data.direction == _direction)]
            temp_s['time_window'] = time_window_
            temp_s['tollgate'] = _id
            temp_s['direction'] = _direction
            temp_s['model0'] = int(list(temp_time_window_data['model']).count(0))
            temp_s['model1'] = int(list(temp_time_window_data['model']).count(1))
            temp_s['model2'] = int(list(temp_time_window_data['model']).count(2))
            temp_s['model3'] = int(list(temp_time_window_data['model']).count(3))
            temp_s['model4'] = int(list(temp_time_window_data['model']).count(4))
            temp_s['model5'] = int(list(temp_time_window_data['model']).count(5))
            temp_s['model6'] = int(list(temp_time_window_data['model']).count(6))
            temp_s['has_etc'] = int(list(temp_time_window_data['is_etc']).count(1))
            temp_s['no_etc'] = int(list(temp_time_window_data['is_etc']).count(0))
            temp_s['veh_type0'] = int(list(temp_time_window_data['veh_type']).count(0))
            temp_s['veh_type1'] = int(list(temp_time_window_data['veh_type']).count(1))
            temp_s['veh_type2'] = int(list(temp_time_window_data['veh_type']).count(2))
            temp_s['pressure'] = temp_weather.iloc[0,2]
            temp_s['sea_pressure'] = temp_weather.iloc[0,3]
            temp_s['wind_direction'] = get_wd(temp_weather.iloc[0,4])
            temp_s['wind_speed'] = temp_weather.iloc[0,5]
            temp_s['temperature'] = temp_weather.iloc[0,6]
            temp_s['rel_humidity'] = temp_weather.iloc[0,7]
            temp_s['precipitation'] = temp_weather.iloc[0,8]
            temp_s['weekday'] = weekday_
            temp_s['date'] = _date_
            temp_s['hour'] = hour_
            temp_s['volumes'] = len(temp_time_window_data)
            volume_collected_test_data = volume_collected_test_data.append(temp_s,ignore_index=True)

            


In [19]:
list(volume_collected_test_data.wind_direction).count('no_direction')
volume_collected_test_data.shape

(2520, 26)

In [20]:
#redifine time window
def count_time(time_window):
    time_ = datetime.strptime(re.split('\[|\ |\,|\)',time_window)[1],'%H:%M:%S')
    return int((time_.hour*3 )+ (time_.minute/20))
    
    
volume_collected_test_data['time_window_'] = volume_collected_test_data['time_window'].apply(count_time)

In [21]:
#redifine date
volume_collected_test_data['date'] = volume_collected_test_data['date'].apply(lambda x: int(re.split('\-',x)[2]))

In [22]:
#redifine wind_direction
def redifine_wd(wind_direction):
    switcher ={
        'N':0,
        'NNE':1,
        'NE':2,
        'ENE':3,
        'E':4,
        'ESE':5,
        'SE':6,
        'SSE':7,
        'S':8,
        'SSW':9,
        'SW':10,
        'WSW':11,
        'W':12,
        'WNW':13,
        'NW':14,
        'NNW':15,
        'N':16,
        'no_direction':17,
    }
    return switcher.get(wind_direction)
volume_collected_test_data['wind_direction'] = volume_collected_test_data['wind_direction'].apply(redifine_wd)

In [25]:
list(volume_collected_test_data.columns)

['time_window',
 'tollgate',
 'direction',
 'model0',
 'model1',
 'model2',
 'model3',
 'model4',
 'model5',
 'model6',
 'has_etc',
 'no_etc',
 'veh_type0',
 'veh_type1',
 'veh_type2',
 'pressure',
 'sea_pressure',
 'wind_direction',
 'wind_speed',
 'temperature',
 'rel_humidity',
 'precipitation',
 'weekday',
 'date',
 'hour',
 'volumes',
 'time_window_']

In [28]:
#drop some variable
volume_droped =volume_collected_test_data.drop(['model0',
 'model1',
 'model2',
 'model3',
 'model4',
 'model5',
 'model6',
 'has_etc',
 'no_etc',
 'veh_type0',
 'veh_type1',
 'veh_type2',
 'pressure',
 'sea_pressure','time_window','hour'],axis=1,inplace=False)


In [29]:
volume_droped.head()

,tollgate,direction,wind_direction,wind_speed,temperature,rel_humidity,precipitation,weekday,date,volumes,time_window_
0,1,0,15,1.9,20.4,87.0,0.0,6,23,34,30
1,1,1,15,1.9,20.4,87.0,0.0,6,23,90,30
2,2,0,15,1.9,20.4,87.0,0.0,6,23,75,30
3,3,0,15,1.9,20.4,87.0,0.0,6,23,98,30
4,3,1,15,1.9,20.4,87.0,0.0,6,23,93,30


### One-Hot Coding

In [31]:
var_to_encode = ['tollgate','direction','wind_direction']

volume_droped = pd.get_dummies(volume_droped, columns=var_to_encode)


In [34]:
#drop columns for test
volume_droped.columns


Index([u'wind_speed', u'temperature', u'rel_humidity', u'precipitation',
       u'weekday', u'date', u'volumes', u'time_window_', u'tollgate_1',
       u'tollgate_2', u'tollgate_3', u'direction_0', u'direction_1',
       u'wind_direction_1', u'wind_direction_2', u'wind_direction_3',
       u'wind_direction_4', u'wind_direction_5', u'wind_direction_6',
       u'wind_direction_7', u'wind_direction_8', u'wind_direction_9',
       u'wind_direction_10', u'wind_direction_14', u'wind_direction_15',
       u'wind_direction_16'],
      dtype='object')

In [35]:
volume_droped['wind_direction_11'] = volume_droped['wind_direction_1'].apply(lambda x:0)
volume_droped['wind_direction_12'] = volume_droped['wind_direction_1'].apply(lambda x:0)
volume_droped['wind_direction_13'] = volume_droped['wind_direction_1'].apply(lambda x:0)
volume_droped['wind_direction_17'] = volume_droped['wind_direction_1'].apply(lambda x:0)

In [38]:
list_ = ['time_window_','date','weekday','tollgate_1',
         'tollgate_2','tollgate_3','direction_0','direction_1',
         'temperature','rel_humidity','precipitation','wind_speed',
         'wind_direction_1','wind_direction_2','wind_direction_3','wind_direction_4',
         'wind_direction_5','wind_direction_6','wind_direction_7','wind_direction_8',
         'wind_direction_9','wind_direction_10','wind_direction_11','wind_direction_12',
         'wind_direction_13','wind_direction_14','wind_direction_15','wind_direction_16',
         'wind_direction_17','volumes']


In [39]:
volume_final_test_data = volume_droped[list_]
volume_final_test_data

(2520, 30)

In [40]:
save_root = '/home/chenhr/Documents/KDD2017/dataSets/dataSets/training/'
volume_final_test_data.to_csv(save_root + 'train_data/test_modified.csv',index=False)


In [41]:
volume_final_test_data


,time_window_,date,weekday,tollgate_1,tollgate_2,tollgate_3,direction_0,direction_1,temperature,rel_humidity,...,wind_direction_9,wind_direction_10,wind_direction_11,wind_direction_12,wind_direction_13,wind_direction_14,wind_direction_15,wind_direction_16,wind_direction_17,volumes
0,30,23,6,1,0,0,1,0,20.4,87.0,...,0,0,0,0,0,0,1,0,0,34
1,30,23,6,1,0,0,0,1,20.4,87.0,...,0,0,0,0,0,0,1,0,0,90
2,30,23,6,0,1,0,1,0,20.4,87.0,...,0,0,0,0,0,0,1,0,0,75
3,30,23,6,0,0,1,1,0,20.4,87.0,...,0,0,0,0,0,0,1,0,0,98
4,30,23,6,0,0,1,0,1,20.4,87.0,...,0,0,0,0,0,0,1,0,0,93
5,3,18,1,1,0,0,1,0,20.5,87.0,...,0,0,0,0,0,0,0,0,0,11
6,3,18,1,1,0,0,0,1,20.5,87.0,...,0,0,0,0,0,0,0,0,0,13
7,3,18,1,0,1,0,1,0,20.5,87.0,...,0,0,0,0,0,0,0,0,0,0
8,3,18,1,0,0,1,1,0,20.5,87.0,...,0,0,0,0,0,0,0,0,0,23
9,3,18,1,0,0,1,0,1,20.5,87.0,...,0,0,0,0,0,0,0,0,0,10
